# Imported functions

In [41]:
import sympy as sp
import itertools
import pypolsys
import numpy as np
from itertools import chain, combinations, product
from typing import List, Iterator, Tuple, Any, Generator
import time

# Defining the game class and a function for creating a game

In [42]:
class Game:
    """
    A class used to represent a game in strategic form.

    ...

    Attributes
    ----------
    num_players : int
        number of players in the game
    num_strategies : list
        number of strategies for each player
    payoffs : list
        payoff matrix of the game
    indices : list
        indices of the strategies for each player in the payoff matrix

    Methods
    -------
    create_n_player_game(payoffs)
        creates a game with n players and variable strategies for each player
    get_strategy_indices(payoffs)
        returns the indices of the strategies for each player
    """
    def __init__(self, num_players, num_strategies, payoffs, indices, probas):
        self.num_players = num_players
        self.num_strategies = num_strategies
        self.payoffs = payoffs
        self.index = indices
        self.proba = probas


def create_n_player_game(payoffs):
    num_players = len(payoffs)
    num_strategies = [len(payoffs[i]) for i in range(num_players)]
    indices = get_strategy_indices(payoffs)
    probas = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(num_strategies[i])] for i in range(num_players)]

    return Game(num_players, num_strategies, payoffs, indices, probas)

def get_strategy_indices(payoffs):
    strategy_indices = []

    for player_payoffs in payoffs:
        player_strategy_indices = []
        for i, strategy in enumerate(player_payoffs):
            player_strategy_indices.append(i)
        strategy_indices.append(tuple(player_strategy_indices))

    return tuple(strategy_indices)


In [43]:
def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list

def list_dig(list, down):
    down = down - 1
    if down > 0:
        list, down = list_dig(list[0], down)
    return list, down


# def is_valid_game_payoffs(payoffs):
#     num_players = len(payoffs)
#     num_strategies = [len(payoffs[i]) for i in range(num_players)]

#     if not payoffs or not all(isinstance(p, list) and p for p in payoffs):
#         raise TypeError("Only nested list structures are allowed")
    
#     for payoff in flatten_list(payoffs):
#         if not isinstance(payoff, (int, float)):
#             raise TypeError("Only integers are allowed")
           
#     for p in range(num_players):
#         print(p, payoffs[p])
#         for player in [player for player in range(num_players) if player != p]:
#             for s in list_dig(payoffs[player], p )[0]:
#                 print(player, s)
#                 if len(s) != len(payoffs[p]):
#                     raise ValueError("Player", p, "and", player, "have inconsistent payoff matrix dimensions")
                    
#     return print("Payoffs valid for game with", num_players, "players and", num_strategies, "strategies")


# Different games

In [44]:
payoffs = [
    [
        [
            [9, 0],
            [0, 3]
        ],
        [
            [0, 3],
            [9, 0]
        ]
    ],
    [
        [
            [8, 0],
            [0, 4]
        ],
        [
            [0, 4],
            [8, 0]
        ]
    ],
    [
        [
            [12, 0],
            [0, 2]
        ],
        [
            [0, 6],
            [6, 0]
        ]
    ]
    ]

game = create_n_player_game(payoffs)
game.num_players, game.num_strategies#, game.payoffs

(3, [2, 2, 2])

In [45]:
payoffs_test = [
    [
        [
            [9, 0],
            [0, 3]
        ],
        [
            [-10, -3],
            [-9, -10]
        ]
    ],
    [
        [
            [8, 0],
            [0, 4]
        ],
        [
            [0, 4],
            [8, 0]
        ]
    ],
    [
        [
            [12, 0],
            [0, 2]
        ],
        [
            [0, 6],
            [6, 0]
        ]
    ]
    ]

game_t = create_n_player_game(payoffs_test)

In [46]:
payoffs21 = [
    [
        [1,0],
        [0,1]
    ],
    [
        [1,0],
        [0,1]
    ]
]

game21 = create_n_player_game(payoffs21)
game21.proba

[[p11, p12], [p21, p22]]

In [47]:
payoffs22 = [
    [
        [1,0],
        [0,1],
        [-1,-1]
    ],
    [
        [1,0,4],
        [0,1,3]
    ]
]

game22 = create_n_player_game(payoffs22)

In [48]:
payoffs2 = [
    [
        [
            1,
            -1,
            -2
        ],
        [
            -3,
            1,
            -4
        ],
        [
            -2,
            0,
            1
        ]
    ],
    [
        [
            1,
            0,
            -1
        ],
        [
            -1,
            1,
            0
        ],
        [
            0,
            -1,
            1
        ]
    ]
]

game2 = create_n_player_game(payoffs2)
game2.num_players, game2.num_strategies#, game2.payoffs

(2, [3, 3])

In [49]:
payoffs3 = [
    [
        [
            [9, 0],
            [0, 3]
        ],
        [
            [0, 3],
            [9, 0]
        ]
        ,
        [
            [-1, 1],
            [2, 4]
        ]
    ],
    [
        [
            [8, 0],
            [0, 4],
            [2, 3]
        ],
        [
            [0, 4],
            [8, 0],
            [5, 7]
        ]
    ],
    [
        [
            [12, 0],
            [0, 2],
            [1, 1]
        ],
        [
            [0, 6],
            [6, 0],
            [2, 3]
        ]
    ]
]

game3 = create_n_player_game(payoffs3)
game3.num_players, game3.num_strategies#, game3.payoffs

(3, [3, 2, 2])

In [50]:
payoffs4 = [
    [
        [[[1, 2], [3, 4]], [[5, 6], [7, 8]]],
        [[[9, 10], [11, 12]], [[13, 14], [15, 16]]]
    ],
    [
        [[[17, 18], [19, 20]], [[21, 22], [23, 24]]],
        [[[25, 26], [27, 28]], [[29, 30], [31, 32]]]
    ],
    [
        [[[33, 34], [35, 36]], [[37, 38], [39, 40]]],
        [[[41, 42], [43, 44]], [[45, 46], [47, 48]]]
    ],
    [
        [[[49, 50], [51, 52]], [[53, 54], [55, 56]]],
        [[[57, 58], [59, 60]], [[61, 62], [63, 64]]]
    ]
]
game4 = create_n_player_game(payoffs4)
game4.num_players, game4.num_strategies#, game4.payoffs

(4, [2, 2, 2, 2])

# Functions for finidng all Nash equilibria with full support in a game

In [51]:
def calculate_expected_payoff_diff_equations(game, probabilities):
    n = game.num_players

    def expected_payoff(player, strategy):
        s = game.num_strategies[player]
        payoff = 0

        # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
        strategy_ranges = [range(game.num_strategies[p]) if p != player else [strategy] for p in range(n)]
        for indices in itertools.product(*strategy_ranges):
            probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(n)]
            payoff_value = game.payoffs[player][strategy]
            for i, index in enumerate(indices):
                if i != player:
                    payoff_value = payoff_value[index]
            payoff_term = payoff_value * sp.prod(probability_combination)

            payoff += payoff_term
        return payoff

    payoff_differences_equations = []
    for player in range(n):
        player_payoff_diff = []
        for strategy in range(game.num_strategies[player]-1):
            diff = expected_payoff(player, strategy) - expected_payoff(player, game.num_strategies[player]-1)
            player_payoff_diff.append(diff)
        payoff_differences_equations.append(player_payoff_diff)

    # Flatten the list of lists
    flat_payoff_differences_equations = [diff for sublist in payoff_differences_equations for diff in sublist]

    return flat_payoff_differences_equations

In [52]:
def calculate_payoff_diff_and_prob_sum_equations(game):
    n = game.num_players
    # Create symbolic probability variables
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Collect the equations in a list
    equations = []
    for player_payoff_diff in calculate_expected_payoff_diff_equations(game, probabilities):
        equations.append(sp.poly(player_payoff_diff , *[p for player_probs in probabilities for p in player_probs]))

    for player_probs in probabilities:
        prob_sum_eq = sum(player_probs) - 1
        equations.append(sp.poly(prob_sum_eq, *[p for player_probs in probabilities for p in player_probs]))


    return equations

In [53]:
def nash_solver(game, tol):
    D = sum(game.num_strategies)
    pol = pypolsys.utils.fromSympy(calculate_payoff_diff_and_prob_sum_equations(game))
    pypolsys.polsys.init_poly(*pol)
    part = pypolsys.utils.make_h_part(D)
    pypolsys.polsys.init_partition(*part)
    bplp = pypolsys.polsys.solve(1e-8, 1e-15, 0.0)
    r = pypolsys.polsys.myroots
    def remove_near_zero_complex(array, tol):
        # Check if the imaginary or real part of any element in a row is close to zero
        is_complex_close_to_zero = lambda row: any(x.real < tol or abs(x.imag) > tol for x in row)
    
        # Create a mask to filter out rows with complex elements close to zero
        mask = np.array([not is_complex_close_to_zero(row) for row in array])
    
        # Return the real parts of the filtered array
        return np.real(array[mask])
    
    def subset_list(input_list, lengths):
        sublists = []
        start = 0

        for length in lengths:
            sublists.append(input_list[start:start + length])
            start += length

        return sublists
    
    ok_sol = remove_near_zero_complex(r[:D,:].transpose(), tol)
    output = []
    for sol in ok_sol:
        output.append(subset_list(sol, game.num_strategies))        
    
    return output

# Functions for generating block games in right order

In [54]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1))

In [55]:
def potential_support_pairs(game):
    num_strategies = game.num_strategies
    support_sets = [sorted(list(powerset(range(num))), key=lambda x: (len(x), x)) for num in num_strategies]

    combined_supports_list = list(product(*support_sets))
    sorted_combined_supports_list = sorted(combined_supports_list, key=lambda x: (sum(len(y) for y in x), max(len(y) for y in x) - min(len(y) for y in x), x))

    for sorted_combined_supports in sorted_combined_supports_list:
        yield sorted_combined_supports

In [56]:
def payoff_player(payoff, indices):
    #print("Payoffs:", payoff, "Indices:", indices)
    indices = list(indices)
    index = indices.pop(0)
    t_payoff = [payoff[i] for i in index]
    for i, pay in enumerate(t_payoff):
        if len(indices)>0:
            pay_1 = payoff_player(pay, indices)
            t_payoff[i] = pay_1
        else:
            t_payoff[i] = pay
    return t_payoff

In [57]:
def block_game(payoffs, indices):
    block = []
    for p, payoff in enumerate(payoffs):
        index =  [*indices]
        index.pop(p)
        indexes = tuple([indices[p], *index])
        block_n = payoff_player(payoff, indexes)
        block.append(block_n)
    return block

# Functions for determing whether a pure strategy is conditionally dominated

In [58]:
def is_strategy_strictly_dominated(payoffs, player, strategy):
    num_players = len(payoffs)
    num_strategies = [len(payoffs[i]) for i in range(num_players)]
    other_strategies = [i for i in range(num_strategies[player]) if i != strategy]

    def get_payoff(payoffs, indices, player, strategy):
        payoff = payoffs[player][strategy]
        for i, index in enumerate(indices):
            if i != player:
                payoff = payoff[index]
        return payoff
    
    for other_strategy in other_strategies:
        temp = 0
        temp2 = 0
        for indices in itertools.product(*[range(num_strategies[p]) if p != player else other_strategies for p in range(num_players)]):
            temp2 += 1
            payoff_current_strategy = get_payoff(payoffs, indices, player, strategy)

            payoff_other_strategy = get_payoff(payoffs, indices, player, other_strategy)
            
            if payoff_current_strategy > payoff_other_strategy:
                break
            else:
                temp += 1
        if temp == temp2: return True
            
    return False


In [59]:
def admissible_block(payoffs):
    num_players = len(payoffs)
    num_strategies = [len(payoffs[i]) for i in range(num_players)]
    for p in range(num_players):
        for s in range(num_strategies[p]):
            if is_strategy_strictly_dominated(payoffs, p, s):
                return False
    return True

# Functions to check whether a completely mixed Nash equilibrium of block game is Nash in larger game

In [60]:
num_players = len(payoffs21)
num_strategies = [len(payoffs21[i]) for i in range(num_players)]
probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(num_strategies[i])] for i in range(num_players)]

def expected_payoff(player, strategy):

    payoff = 0

    # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
    strategy_ranges = [range(num_strategies[p]) if p != player else [strategy] for p in range(num_players)]
    for indices in itertools.product(*strategy_ranges):
        probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(num_players)]
        payoff_value = payoffs21[player][strategy]

        for i, index in enumerate(indices):
            if i != player:
                payoff_value = payoff_value[index]     
        payoff_term = payoff_value * sp.prod(probability_combination)

        payoff += payoff_term
    return payoff

In [61]:
def is_strategy_better(payoffs, mixed_strategy_profile, player, new_strategy):
    num_players = len(payoffs)
    num_strategies = [len(payoffs[i]) for i in range(num_players)]
    # Create symbolic probability variables
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(num_strategies[i])] for i in range(num_players)]
    
    def expected_payoff(player, strategy):

        payoff = 0

        # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
        strategy_ranges = [range(num_strategies[p]) if p != player else [strategy] for p in range(num_players)]
        for indices in itertools.product(*strategy_ranges):
            probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(num_players)]
            payoff_value = payoffs[player][strategy]
            for i, index in enumerate(indices):
                if i != player:
                    payoff_value = payoff_value[index]
            payoff_term = payoff_value * sp.prod(probability_combination)

            payoff += payoff_term
        return payoff

    def expected_payoff_from_mixed_strategy(player):
        total_payoff = 0
        for strategy in range(num_strategies[player]):
            total_payoff += probabilities[player][strategy] * expected_payoff(player, strategy)
        return total_payoff

    # Calculate the current expected payoff
    current_expected_payoff = expected_payoff_from_mixed_strategy(player)
    potential_expected_payoff = expected_payoff(player, new_strategy)
    for p in range(num_players):
        current_expected_payoff = current_expected_payoff.subs([(probabilities[p][i], mixed_strategy_profile[p][i]) for i in range(num_strategies[p])])
    # Calculate the expected payoff if the player switches to the new strategy
        potential_expected_payoff = potential_expected_payoff.subs([(probabilities[p][i], mixed_strategy_profile[p][i]) for i in range(num_strategies[p])])
    # Check if the new strategy is better
    return potential_expected_payoff > current_expected_payoff

In [62]:
def stra_em(ne_cand, block_index, index):
    full_sp = [
        [ne_cand[p][block_index[p].index(s)] if s in block_index[p] else 0 for s in index[p]]
        for p in range(len(index))
    ]
    return full_sp

# Nash equilibrium solver

In [64]:
def ne_solver(game): #, index = False
    result = []

    for indices in potential_support_pairs(game):
        block = block_game(game.payoffs, indices)
        bg = create_n_player_game(block)

        skip_indices = False
        for p in range(game.num_players):
            if skip_indices:
                break
            for s in range(bg.num_strategies[p]):
                if is_strategy_strictly_dominated(bg.payoffs, p, s):
                    skip_indices = True
                    break
                    
        skip_eq = False
        pot_sol = nash_solver(bg, 1e-6)
        
        for sol in pot_sol:
            m_sp = stra_em(sol, indices, game.index)           
            for p in range(game.num_players):
                if skip_eq:
                    break
                for s in set(game.index[p]).difference(set(indices[p])):
                    if is_strategy_better(game.payoffs, m_sp, p ,s):
                        skip_eq = True
                        break
            if skip_eq:
                break
            result.append(m_sp)

    return result

# Functions for calculating index of Nash equilibria in generic game

In [63]:
def calculate_expected_payoff_diff_equations_fj(game):
    n = game.num_players
    # Create symbolic probability variables
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    def expected_payoff(player, strategy):
        s = game.num_strategies[player]
        payoff = 0

        # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
        strategy_ranges = [range(game.num_strategies[p]) if p != player else [strategy] for p in range(n)]
        for indices in itertools.product(*strategy_ranges):
            probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(n)]
            payoff_value = game.payoffs[player][strategy]
            for i, index in enumerate(indices):
                if i != player:
                    payoff_value = payoff_value[index]
            payoff_term = payoff_value * sp.prod(probability_combination)

            payoff += payoff_term
        return payoff

    def expected_payoff_from_mixed_strategy(player):
        s = game.num_strategies[player]
        total_payoff = 0
        for strategy in range(s):
            total_payoff += probabilities[player][strategy] * expected_payoff(player, strategy)
        return total_payoff

    payoff_differences_equations = []
    for player in range(n):
        player_payoff_diff = []
        for strategy in range(game.num_strategies[player]):
            diff = probabilities[player][strategy] * (expected_payoff(player, strategy) - expected_payoff_from_mixed_strategy(player))
            player_payoff_diff.append(diff)
        payoff_differences_equations.append(player_payoff_diff)

    # Flatten the list of lists
    flat_payoff_differences_equations = [diff for sublist in payoff_differences_equations for diff in sublist]

    return flat_payoff_differences_equations

In [65]:
def calculate_index(game, mix_sp):
    # Get the flattened payoff difference equations
    payoff_differences_equations = calculate_expected_payoff_diff_equations_fj(game)
    
    n = game.num_players
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Flatten the list of probability variables
    flat_probabilities = [p for sublist in probabilities for p in sublist]

    # Calculate the Jacobian
    jacobian_matrix = sp.Matrix(payoff_differences_equations).jacobian(flat_probabilities)
    for p in range(n):
        jacobian_matrix = jacobian_matrix.subs([(probabilities[p][i], mix_sp[p][i]) for i in range(game.num_strategies[p])])
    output = np.array(jacobian_matrix).astype(np.float64)
    return np.sign((-1)**(sum(game.num_strategies))) * np.sign(np.linalg.det(output))

In [66]:
def rounder(sol, digit):
    return [np.round(np.array(sublist).astype(np.float64), digit) for sublist in sol]

In [67]:
for sol in ne_solver(game22):
    index = calculate_index(game22, sol)
    print("NE:", rounder(sol,3), "Index:", index)

NE: [array([1., 0., 0.]), array([1., 0.])] Index: 1.0
NE: [array([0., 1., 0.]), array([0., 1.])] Index: 1.0
NE: [array([0.5, 0.5, 0. ]), array([0.5, 0.5])] Index: -1.0


# Functions for finding minimal game blocks

In [68]:
def are_nested_indices(index_set1, index_set2):
    for set1, set2 in zip(index_set1, index_set2):
        if not set(set1).issubset(set2):
            return False
    return True

In [82]:
def min_game_blocks(game):
    
    min_gbs = []
    
    solid_outcomes = []
    
    bg_ne_index = []
    
    def ne_index(pot_sol):
        
        ne_and_index = []
        skip_eq = False
        
        for sol in pot_sol:
            m_sp = stra_em(sol, indices, game.index)           
            for p in range(game.num_players):
                if skip_eq:
                    break
                for s in set(game.index[p]).difference(set(indices[p])):
                    if is_strategy_better(game.payoffs, m_sp, p ,s):
                        skip_eq = True
                        break
            if skip_eq:
                break
            index = calculate_index(game,m_sp)
            ne_and_index.append([rounder(m_sp,3), index])
                
        return ne_and_index
        
    for indices in potential_support_pairs(game):
        
        block = block_game(game.payoffs, indices)
        bg = create_n_player_game(block)
        
        nmgb = False
        
        for mgb in min_gbs:
            if are_nested_indices(mgb,indices):
                nmgb = True
                break
        if nmgb:
            continue
            
        skip_indices = False

        for p in range(game.num_players):
            if skip_indices:
                break
            for s in range(bg.num_strategies[p]):
                if is_strategy_strictly_dominated(bg.payoffs, p, s):
                    skip_indices = True
                    break
        if skip_indices:
            continue
            
        pot_sol = nash_solver(bg, 1e-6)
        nei = ne_index(pot_sol)
        if nei:
            bg_ne_index.append([indices, nei])
        
        index_counter = 0

        block_list = [bg[1] for bg in bg_ne_index if are_nested_indices(bg[0], indices)]     
        for i in block_list:
            index_counter += sum([nei[1] for nei in i])
        if index_counter == 1:
            min_gbs.append(indices)
            solid_outcomes.append(block_list)
    return zip(min_gbs,solid_outcomes)

In [70]:
for mgb in min_game_blocks(game):
    print(mgb)

(((0,), (0,), (0,)), [[[[array([1., 0.]), array([1., 0.]), array([1., 0.])], 1.0]]])
(((0,), (1,), (1,)), [[[[array([1., 0.]), array([0., 1.]), array([0., 1.])], 1.0]]])
(((1,), (0,), (1,)), [[[[array([0., 1.]), array([1., 0.]), array([0., 1.])], 1.0]]])
(((1,), (1,), (0,)), [[[[array([0., 1.]), array([0., 1.]), array([1., 0.])], 1.0]]])


In [71]:
def generate_payoffs(num_strategies):
    payoffs = []
    num_players = len(num_strategies)
    # Generate payoffs for each player
    for player in range(num_players):
        player_payoffs = []

        # Generate payoffs for each strategy combination
        strategy_combinations = np.product(num_strategies)
        player_payoffs = np.random.rand(strategy_combinations).reshape(*num_strategies).tolist()

        # Assign payoffs to the player
        payoffs.append(player_payoffs)

    return payoffs

In [75]:
random_payoffs= generate_payoffs([4,4,4])

In [76]:
random_game = create_n_player_game(random_payoffs)
for mgb in min_game_blocks(random_game):
    print(mgb)

(((0, 1), (0,), (1, 3)), [[[[array([0.79, 0.21, 0.  , 0.  ]), array([1., 0., 0., 0.]), array([0.   , 0.643, 0.   , 0.357])], 1.0]]])
(((0, 3), (2, 3), (1,)), [[[[array([0.214, 0.   , 0.   , 0.786]), array([0.   , 0.   , 0.443, 0.557]), array([0., 1., 0., 0.])], 1.0]]])
(((1,), (0, 1), (1, 2)), [[[[array([0., 1., 0., 0.]), array([0.473, 0.527, 0.   , 0.   ]), array([0.   , 0.111, 0.889, 0.   ])], 1.0]]])
(((2,), (1, 2), (0, 3)), [[[[array([0., 0., 1., 0.]), array([0.   , 0.378, 0.622, 0.   ]), array([0.037, 0.   , 0.   , 0.963])], 1.0]]])
(((3,), (2, 3), (1, 2)), [[[[array([0., 0., 0., 1.]), array([0.   , 0.   , 0.432, 0.568]), array([0.   , 0.012, 0.988, 0.   ])], 1.0]]])


In [85]:
n = 200

n_ne = 0
tic = time.perf_counter()
list_ne = []
for i in range(n):
    current_ne = 0
    random_payoffs= generate_payoffs([,4,4,4])
    random_game = create_n_player_game(random_payoffs)
    for mgb in min_game_blocks(random_game):
        n_ne +=  len(mgb[1])
        current_ne += len(mgb[1])
    list_ne.append(current_ne)
    if i%1==0:
        toc = time.perf_counter()
        print(np.round(toc -tic,1) , i / n, "Nr SO:", n_ne / (i+1), list_ne )
        
n_ne / n, list_ne

KeyboardInterrupt: 

In [81]:
n = 200

n_ne = 0
tic = time.perf_counter()
for i in range(n):
    random_payoffs= generate_payoffs([5,5,5])
    random_game = create_n_player_game(random_payoffs)
    for mgb in min_game_blocks(random_game):
        n_ne +=  len(mgb[1])
    toc = time.perf_counter()
    print(np.round(toc -tic,1) , i / n, "Nr SO:", n_ne / (i+1) )
        
n_ne / n

4494.1 0.0 Nr SO: 7.0
8123.9 0.005 Nr SO: 7.0
12522.6 0.01 Nr SO: 7.333333333333333
17384.9 0.015 Nr SO: 6.0
21367.9 0.02 Nr SO: 5.8
24788.3 0.025 Nr SO: 7.333333333333333
26858.3 0.03 Nr SO: 9.285714285714286
29621.5 0.035 Nr SO: 8.75
31327.9 0.04 Nr SO: 8.555555555555555
36797.9 0.045 Nr SO: 8.4


KeyboardInterrupt: 

In [ ]:
random_payoffs= generate_payoffs([4,4,4,4])
random_game = create_n_player_game(random_payoffs)
for mgb in min_game_blocks(random_game):
    print(mgb)